In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Modeling Non-linear Relationships

In this notebook, we will use basic feature transformations (feature engineering) to model non-linear relationships using linear models.

## Toy Data set

To enable easy visualization of the model fitting process we will use a simple synthetic data set.

In [ ]:
data = pd.read_csv("data/synthetic_data.csv")
data.head()

We can visualize the data in three dimensions:

In [ ]:
data_scatter = go.Scatter3d(x=data["X0"], y=data["X1"], z=data["Y"], 
                            mode="markers",
                            marker=dict(size=2))
layout = dict(margin=dict(l=0, r=0, t=0, b=0), 
              height=600,
              scene = dict(xaxis_title='X0', yaxis_title='X1', zaxis_title='Y'))
go.Figure([data_scatter], layout)

## Basic Linear Model

We normally start with a basic linear model with an intercept term.

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(data[["X0", "X1"]], data[["Y"]])

To track the performance of our models, we use the following plotting functions.

In [ ]:
def make_surface(predict, grid_points = 30, name="Linear Model"):
    """
    This function constructs a 3d surface from a prediction function.
    """
    u = np.linspace(data["X0"].min(), data["X0"].max(), grid_points)
    v = np.linspace(data["X1"].min(), data["X1"].max(), grid_points)
    xu, xv = np.meshgrid(u,v)
    X = np.vstack((xu.flatten(),xv.flatten())).transpose()
    z = predict(X)
    return go.Surface(x=xu, y=xv, z=z.reshape(xu.shape), opacity=0.8, 
                      name=name, showscale=False)


def evaluate_model(predict, grid_points=30):  
    """
    This function visualizes the predict function 
    """
    # Compute Y and Yhat
    Y = data["Y"].to_numpy()
    Yhat = predict(data[["X0", "X1"]].to_numpy()).flatten()
    # Compute and print error metrics
    print("Mean Absolute Error:", mean_absolute_error(Y, Yhat))
    print("Root Mean Squared Error:", np.sqrt(mean_squared_error(Y, Yhat)))

    # Make Side by Side Plots
    fig = make_subplots(rows=1, cols=2, 
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig.add_trace(data_scatter, row=1, col=1)
    fig.add_trace(make_surface(predict, grid_points=grid_points), row=1, col=1)
    fig.update_layout(layout)
    fig.add_trace(go.Scatter(x=Yhat, y=Y, mode="markers"), row=1, col=2)
    ymin = np.min([np.min(Y), np.min(Yhat)])
    ymax = np.max([np.max(Y), np.max(Yhat)])
    fig.add_trace(go.Scatter(x=[ymin,ymax], y=[ymin,ymax], name="y=yhat"))
    fig.update_layout(xaxis_title ="yhat", yaxis_title="Y")
    fig.update_layout(showlegend=False)
    fig.show()


Examining our latest model:

In [ ]:
evaluate_model(model.predict, grid_points=5)

Examining the above data we see that there is some **periodic** structure as well as some **curvature**. Can we fit this data with a linear model?

## What does it mean to be a _Linear Model_

Linear models are **linear combinations** of features.  These models are therefore linear in the **parameters** but not necessarily the underlying data.  We can encode non-linearity in our data through the use of feature functions:


$$
f_\theta\left( x \right) = \phi(x)^T \theta = \sum_{j=0}^{p} \phi(x)_j \theta_j
$$

where $\phi$ is an *arbitrary function* from $x\in \mathbb{R}^d$ to $\phi(x) \in \mathbb{R}^{p+1}$. We could also denote these as a collection of separate feature $\phi_j$ feature functions from $x\in \mathbb{R}^d$ to $\phi_j(x) \in \mathbb{R}$:

$$
\phi(x) = \left[\phi_0(x), \phi_1(x), \ldots, \phi_p(x) \right]
$$


We often refer to these $\phi_j$ as **feature functions** and their design plays a critical role in both how we capture prior knowledge and our ability to fit complicated data

## Introducing Non-linear Features

In the following, we will add a few different sine functions at different frequencies and offsets.

$$
\sin\left(2 \pi * \textbf{frequency}X + \textbf{phase}\right)
$$

Note that for this to remain a linear model, we cannot make the frequency or phase of the sine function a model parameter.  In fact, these are actually **hyperparameters** of the model that would need to be tuned using either domain knowledge or other search procedures. 

In [ ]:
def phi_periodic(X):
    return np.hstack([
        X,
        np.sin(X),
        np.sin(10*X),
        np.sin(X + 1),
        np.sin(10*X + 1),
    ])
    

In [ ]:
Phi = phi_periodic(data[["X0", "X1"]])
Phi

In [ ]:
Phi.shape

In [ ]:
model_phi = LinearRegression()
model_phi.fit(Phi, data["Y"])

Notice that to make predictions I need to actually apply the $\Phi$ feature function to my data.

In [ ]:
evaluate_model(lambda X: model_phi.predict(phi_periodic(X)))

There is still some curvature.  We can introduce additional polynomial terms to try to improve the fit of our model.

In [ ]:
def phi_curved(X):
    return np.hstack([
        X * X,
        np.expand_dims(np.prod(X, axis=1), 1),
        X ** 3,
    ])

Can you guess the new number of features?

In [ ]:
phi_curved(data[["X0", "X1"]]).shape

Let's build a feature function that combines our features so far:

In [ ]:
def phi_curved_and_periodic(X):
    return np.hstack([phi_curved(X), phi_periodic(X)])

In [ ]:
crazy_model = LinearRegression()
crazy_model.fit(phi_curved_and_periodic(data[["X0", "X1"]]), data[["Y"]])
crazy_model.coef_

In [ ]:
evaluate_model(lambda X: crazy_model.predict(phi_curved_and_periodic(X)))

## Success!!!!!